In [1]:
import json
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib
import operator
import scipy
import math

import numpy as np
import scipy as sp
from  operator import itemgetter
from scipy.stats import norm


In [2]:
dates_new = ["23-04-21", "24-04-21", "25-04-21", "26-04-21", "27-04-21", "28-04-21", "29-04-21", "30-04-21", "01-05-21", "02-05-21", "03-05-21", "04-05-21", "05-05-21", "06-05-21", "07-05-21",
"08-05-21", "09-05-21", "10-05-21"]
week_1_dates = ["23-04-21", "24-04-21", "25-04-21"] 
week_2_dates = ["26-04-21", "27-04-21", "28-04-21", "29-04-21", "30-04-21", "01-05-21", "02-05-21"]
week_3_dates = ["03-05-21", "04-05-21", "05-05-21", "06-05-21", "07-05-21", "08-05-21", "09-05-21"]
week_4_dates = ["10-05-21"]


dir_dataset = 'datasetAprileMaggio/'
files_dataset1 = ['ln_' + str(date) + '.json' for date in week_1_dates]
files_dataset2 = ['ln_' + str(date) + '.json' for date in week_2_dates]
files_dataset3 = ['ln_' + str(date) + '.json' for date in week_3_dates]

dir_channels = 'datasetAprileMaggio/channels/'


In [3]:
dict_dates = {}
i = 0
for filename in files_dataset1:

    with open (dir_channels+'channels_' + filename) as data_file:    
        data_channels = json.load(data_file)
    df_channels = pd.json_normalize(data_channels)
    print("DF_CHANNELS LEN:" ,len(df_channels))

    i+=1

    if i == 1:

        #df_allchannels = df_channels
        for j in range(0,len(df_channels)):
            dict_dates[j]= {'channel_id' : df_channels['channel_id'][j],
                            'day1_node1_htlc' : df_channels['node1_policy.min_htlc'][j],
                            'day1_node1_feebase' : df_channels['node1_policy.fee_base_msat'][j],
                            'day1_node1_feerate' : df_channels['node1_policy.fee_rate_milli_msat'][j],
                            'day1_node2_htlc' : df_channels['node2_policy.min_htlc'][j],
                            'day1_node2_feebase' : df_channels['node2_policy.fee_base_msat'][j],
                            'day1_node2_feerate' : df_channels['node2_policy.fee_rate_milli_msat'][j]
      
                            }

        
        #il file non esiste, quindi sto considerando il primo giorno e devo solo inserire nel file unione tutti i canali del primo file
        display(df_channels.equals(df_allchannels))
    
    else:
        #il file esiste, devo aggiungere i canali che ancora non ho     
        print("Numero di canali nel file unione:", len(df_allchannels))

        #prendo i canali del nuovo dataset che sono anche nel file unione
        in_common = df_channels[df_channels['channel_id'].isin(df_allchannels['channel_id'])]
        print("In comune tra il giorno corrente e il file unione:" ,len(in_common))


        #a questo punto devo prendere i canali aperti del file del giorno, che non si trovano nel grande file, ed aggiungerli
        df_diff_channels = df_channels[~df_channels['channel_id'].isin(df_allchannels['channel_id'])]
        print("Numero di canali diversi nel nuovo file:", len(df_diff_channels))

        #creo il nuovo file mettendo insieme tutti i vecchi + quelli nuovi che ancora non erano nel file unione
        new_df = pd.concat([df_allchannels, df_diff_channels])
        print("Numero di canali totali dopo questa iterazione", len(new_df))

        df_allchannels = new_df



DF_CHANNELS LEN: 19482


KeyError: 0

In [18]:
dict_dates = {}
with open ('datasetAprileMaggio/channels/channels_ln_29-04-21.json') as data_file:    
        data_channels = json.load(data_file)
df_channels = pd.json_normalize(data_channels)

for j in range(0,len(df_channels)):
        dict_dates['channel_id'][j] = df_channels['channel_id'][j]
        dict_dates['day1_node1_htlc'][j] = df_channels['node1_policy.min_htlc'][j]             #'day1_node1_htlc' : [df_channels['node1_policy.min_htlc'][j]],
        dict_dates['day1_node1_feebase'][j] = df_channels['node1_policy.fee_base_msat'][j]                  #'day1_node1_feebase' : df_channels['node1_policy.fee_base_msat'][j],
        dict_dates['day1_node1_feerate'][j] = df_channels['node1_policy.fee_rate_milli_msat'][j]                    #'day1_node1_feerate' : df_channels['node1_policy.fee_rate_milli_msat'][j],
        dict_dates['day1_node2_htlc'][j] = df_channels['node2_policy.min_htlc'][j]               #'day1_node2_htlc' : df_channels['node2_policy.min_htlc'][j],
        dict_dates['day1_node2_feebase'][j] = df_channels['node2_policy.fee_base_msat'][j]                    #'day1_node2_feebase' : df_channels['node2_policy.fee_base_msat'][j],
        dict_dates['day1_node2_feerate'][j] = df_channels['node2_policy.fee_rate_milli_msat'][j]                   #'day1_node2_feerate' : df_channels['node2_policy.fee_rate_milli_msat'][j]
      
                            



KeyError: 'channel_id'

In [27]:
dict_dates = {}
with open ('datasetAprileMaggio/channels/channels_ln_29-04-21.json') as data_file:    
        data_channels = json.load(data_file)
df_channels = pd.json_normalize(data_channels)

for j in range(0,len(df_channels)):
            dict_dates[df_channels['channel_id'][j]] = [
                            df_channels['node1_policy.min_htlc'][j],
                             df_channels['node1_policy.fee_base_msat'][j],
                            df_channels['node1_policy.fee_rate_milli_msat'][j],
                            df_channels['node2_policy.min_htlc'][j],
                             df_channels['node2_policy.fee_base_msat'][j],
                            df_channels['node2_policy.fee_rate_milli_msat'][j]
      
            ]



In [28]:
df = pd.DataFrame(dict_dates)
display(df)

,553951550347608065,554317687705305088,554460624201252865,554494709160148993,554495808645955584,554502405657853952,554574973407920129,554631048514043905,554682725533417473,554819064998199297,...,748544217708101633,748546416775331840,748546416775397377,748546416791126017,748546416799121409,748547516252225536,748548615760314369,748553013810954240,748557411864739841,748559610884194305
0,None,None,None,None,None,None,None,None,None,None,...,1000,1000,1000,1000,1000,1000,1000,1,1000,1000
1,None,None,None,None,None,None,None,None,None,None,...,1000,1000,50,1000,1000,1000,1000,1000,10000,1000
2,None,None,None,None,None,None,None,None,None,None,...,1,1,500,1,1,1,1,10,5000,1
3,None,None,None,None,None,None,None,None,None,None,...,1000,1000,1000,1000,1000,1000,1,1000,1000,1
4,None,None,None,None,None,None,None,None,None,None,...,1000,1000,1000,0,1000,1000,1000,1000,1000,1000
5,None,None,None,None,None,None,None,None,None,None,...,1,1,1,1,1,1,10,1,1,10


In [17]:
df = pd.DataFrame(data=dict_dates)
display(df)

ValueError: If using all scalar values, you must pass an index

In [ ]:
c = p

In [10]:
#for i in range(0,len(df_channels)):
display(df_channels)
display(df_channels['channel_id'][0])

,channel_id,chan_point,last_update,node1_pub,node2_pub,capacity,node1_policy,node2_policy,node1_policy.time_lock_delta,node1_policy.min_htlc,...,node1_policy.disabled,node1_policy.max_htlc_msat,node1_policy.last_update,node2_policy.time_lock_delta,node2_policy.min_htlc,node2_policy.fee_base_msat,node2_policy.fee_rate_milli_msat,node2_policy.disabled,node2_policy.max_htlc_msat,node2_policy.last_update
0,553951550347608065,ede04f9cfc1bb5373fd07d8af9c9b8b5a85cfe5e323b77...,0,03bd3466efd4a7306b539e2314e69efc6b1eaee29734fc...,03c3d14714b78f03fd6ea4997c2b540a4139258249ea1d...,37200,None,None,NaN,None,...,None,None,NaN,NaN,None,None,None,None,None,NaN
1,554317687705305088,cfd0ae79fc150c2c3c4068ceca74bc26652bb269162437...,0,02eccebd9ed98f6d267080a58194dbe554a2b33d976eb9...,02ee4469f2b686d5d02422917ac199602ce4c366a7bfaa...,1000000,None,None,NaN,None,...,None,None,NaN,NaN,None,None,None,None,None,NaN
2,554460624201252865,c0a8d3428f562c232d86be399eb4497934e7e0390fa79e...,0,02eccebd9ed98f6d267080a58194dbe554a2b33d976eb9...,02ee4469f2b686d5d02422917ac199602ce4c366a7bfaa...,1000000,None,None,NaN,None,...,None,None,NaN,NaN,None,None,None,None,None,NaN
3,554494709160148993,06bbac25ed610feb1d07316d1be8b8ba6850ee1dd96cc1...,0,03bd3466efd4a7306b539e2314e69efc6b1eaee29734fc...,03cbf298b068300be33f06c947b9d3f00a0f0e8089da32...,200000,None,None,NaN,None,...,None,None,NaN,NaN,None,None,None,None,None,NaN
4,554495808645955584,2392c45431c064269e4eaeccb0476ac32e56485d84e104...,0,022e74ed3ddd3f590fd6492e60b20dcad7303f17e1ffd8...,02ee4469f2b686d5d02422917ac199602ce4c366a7bfaa...,2000000,None,None,NaN,None,...,None,None,NaN,NaN,None,None,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43326,748547516252225536,3e0261cc57a90d55ee0a75ab752e0cf0025b41144b6c1b...,1619510745,0217890e3aad8d35bc054f43acc00084b25229ecff0ab6...,0251c1c3c167ff1bcdaef55cbfb61cf49f1c0212e3122a...,286582,None,None,40.0,1000,...,False,283717000,1.619511e+09,40.0,1000,1000,1,False,283717000,1.619511e+09
43327,748548615760314369,55863a5c9e87dfc5ab2463f1aad62dd554408a0de40c9a...,1619521490,020dd6a8ec457bc6b005464a7b5d28ea2fb21893305f95...,035736692722a47389d6be965d73a50b1ba63233a1b975...,2000000,None,None,40.0,1000,...,False,2000000000,1.619521e+09,34.0,1,1000,10,False,1980000000,1.619510e+09
43328,748553013810954240,9ab272d8b6f3e8a41bff20600c0fac27b7f1b01d8a0b41...,1619520470,035736692722a47389d6be965d73a50b1ba63233a1b975...,03c8dfbf829eaeb0b6dab099d87fdf7f8faceb0c1b935c...,1000000,None,None,34.0,1,...,False,990000000,1.619514e+09,40.0,1000,1000,1,False,1000000000,1.619520e+09
43329,748557411864739841,86ff1adf6236c84e528233793902acd7d2563986f180fb...,1619518255,02e7c42ae2952d7a71398e23535b53ffc60deb269acbc7...,03179ae29181f2626812b48e11cf30e35cac7b707947bc...,758625,None,None,144.0,1000,...,False,751039000,1.619518e+09,40.0,1000,1000,1,False,751039000,1.619518e+09


'553951550347608065'